# apply filter to images

In [19]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir, min_area_fraction=0.001):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    counter = 1
    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):

        if filename.endswith(('.png', '.jpg', '.jpeg','.tif','.JPG')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define the lower and upper bounds for green color
            lower_green = np.array([27, 30, 30])
            upper_green = np.array([90, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to remove noise
            kernel = np.ones((3, 3), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=1)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            

            # Calculate minimum area based on image dimensions
            min_area = min_area_fraction * image.shape[0] * image.shape[1]

            # Find connected components and their statistics
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # Filter out components based on their area
            for i in range(1, num_labels):  # Exclude background label (0)
                if stats[i, cv2.CC_STAT_AREA] < min_area:
                    mask[labels == i] = 0  # Set pixels of small components to 0

            # Create a white background image
            white_background = np.ones_like(image) * 255

            # Merge the original image and the white background using the mask
            result = cv2.bitwise_and(image, image, mask=mask)
            result += cv2.bitwise_and(white_background, white_background, mask=cv2.bitwise_not(mask))

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, result)
            print(counter)
            counter += 1

# Paths to the directories
input_directory = r"D:\Projects\Weed Detection\mohab data\to be done\sesame\original"
output_directory = r"D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame"

# Call the function with min_area_fraction set to 0.001 (adjust as needed)
detect_green_areas(input_directory, output_directory, min_area_fraction=0.009)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
import cv2
import numpy as np
import os

def extract_and_save_plants(image_path, output_folder, min_contour_area=100):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Image at {image_path} could not be read.")
        return

    # Convert to grayscale and threshold to isolate plants
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contours were found
    if not contours:
        print(f"No contours found in {image_path}")
        return

    # Process each contour
    for i, contour in enumerate(contours):
        # Calculate contour area
        area = cv2.contourArea(contour)

        # Skip small contours
        if area < min_contour_area:
            continue

        # Create a mask for the current contour
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)

        # Apply the mask to the original image to isolate the plant
        masked_image = cv2.bitwise_and(image, image, mask=mask)

        # Make the non-plant areas white instead of black
        masked_white_bg = np.where(mask[:, :, None] == 0, 255, masked_image)

        # Crop the image based on the bounding rectangle of the contour
        x, y, w, h = cv2.boundingRect(contour)
        cropped_image = masked_white_bg[y:y+h, x:x+w]

        # Construct output filename
        output_filename = f'{os.path.splitext(os.path.basename(image_path))[0]}_plant_{i + 1}.png'
        output_path = os.path.join(output_folder, output_filename)

        # Save the cropped image
        cv2.imwrite(output_path, cropped_image)
        print(f"Saved plant image to {output_path}")

def process_folder(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            extract_and_save_plants(image_path, output_folder)

# Example usage
input_folder_path = r"D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame"
output_folder_path = r"D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split"
process_folder(input_folder_path, output_folder_path)


Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1026_class_0_0_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_10_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_11_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_2_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_3_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1028_class_0_4_plant_1.png
Saved 

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1564_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1630_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_164_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1693_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1722_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1750_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_1771_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_177_class_0_0_plant_1.png
Saved plan

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2322_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2323_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2330_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2332_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2363_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_237_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_237_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2405_class_0_0_plant_1.png
Saved plan

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_10_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_11_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_11_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_12_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_2_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_3_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_4_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_2792_class_0_5_plant_1.png
Save

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3160_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3160_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_318_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_318_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3200_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3203_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3226_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_3229_class_0_0_plant_1.png
Saved plan

...

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_10_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_11_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_12_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_13_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_14_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7668_class_0_15_plant_1.png
Sa

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_7997_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8007_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8029_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8029_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8034_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8034_class_0_0_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8034_class_0_0_plant_3.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8034_class_0_1_plant_1.png
Saved pl

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8489_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8493_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8556_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8565_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8565_class_0_0_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8565_class_0_0_plant_3.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8565_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_8565_class_0_1_plant_2.png
Saved pl

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9109_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9126_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9127_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9132_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9132_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9132_class_0_1_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9144_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9145_class_0_0_plant_1.png
Saved pl

Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9601_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9608_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9617_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9617_class_0_0_plant_2.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9618_class_0_0_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9618_class_0_1_plant_1.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9660_class_0_0_plant_7.png
Saved plant image to D:\Projects\Weed Detection\mohab data\to be done\sesame\sesame split\agri_0_9698_class_0_0_plant_1.png
Saved pl